In [37]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

In [38]:
# Image Size
img_size = 224

# Define the function to load and process images
def load_and_process_images(directory, classes, img_size):
    all_arrays = []
    for class_num, faceshape_class in enumerate(classes):
        path = os.path.join(directory, faceshape_class)
        print(f"Loading images from {path}")
        if not os.path.exists(path):
            print(f"Directory {path} does not exist.")
            continue
        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)
                img_array = cv2.imread(img_path)
                if img_array is not None:
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                    img_array = cv2.resize(img_array, (img_size, img_size))
                    all_arrays.append([img_array, class_num])
                else:
                    print(f"Image {img_path} is None")
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")
    return np.array(all_arrays, dtype=object)


In [39]:
# Define the directories and classes
train_dir = "C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Training"
test_dir = "C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing"
classes = ["oval", "rectangular", "round", "square"]

In [40]:
# Load and process images
train_data = load_and_process_images(train_dir, classes, img_size)
test_data = load_and_process_images(test_dir, classes, img_size)

Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Training\oval
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Training\rectangular
Directory C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Training\rectangular does not exist.
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Training\round
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Training\square
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing\oval
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing\rectangular
Directory C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing\rectangular does not exist.
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing\round
Loading images from C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing\square


In [41]:
# Separate features and labels
x_train = np.array([i[0] for i in train_data])
y_train = to_categorical(np.array([i[1] for i in train_data]), num_classes=len(classes))
x_test = np.array([i[0] for i in test_data])
y_test = to_categorical(np.array([i[1] for i in test_data]), num_classes=len(classes))

In [42]:
# Print shapes of datasets
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (240, 224, 224, 3)
y_train shape: (240, 4)
x_test shape: (60, 224, 224, 3)
y_test shape: (60, 4)


In [43]:
# Initialize the model
base_model = VGG16(include_top=False, input_shape=(img_size, img_size, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=20, zoom_range=0.2, shear_range=0.2)
datagen.fit(x_train)

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=60), epochs=50, validation_data=(x_test, y_test))


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [44]:
# Save the model
model.save('girlFacevgg13.h5')

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
# Load the model and make a prediction
model = tf.keras.models.load_model('girlFacevgg13.h5')

# Define the image path and load the image
img_path = 'C:/Users/user/Downloads/Data settt/Girl/Taining dan testing/Testing/oval/img_no_203.jpg'
img = image.load_img(img_path, target_size=(224, 224))

# Convert image to array
input_img = image.img_to_array(img)
input_img = np.expand_dims(input_img, axis=0)
input_img = preprocess_input(input_img)

# Predict the inputs on the model
predict_img = model.predict(input_img)

# Debugging: Print prediction probabilities and indices
print(f"Prediction probabilities: {predict_img}")

# Define your list of face shape classes
classes = ["oval", "rectangular", "round", "square"]

# Map predicted class indices to class labels
predicted_class_indices = np.argmax(predict_img, axis=1)
print(f"Predicted class indices: {predicted_class_indices}")

# Ensure predicted class indices are within range of classes list
predicted_classes = []
for idx in predicted_class_indices:
    if 0 <= idx < len(classes):
        predicted_classes.append(classes[idx])
    else:
        predicted_classes.append("Unknown")  # Handle out-of-range indices gracefully

# Print predicted face shape
print(f"Predicted face shape: {predicted_classes[0]}")

1/1 [==============================] - 1s 1s/step
Prediction probabilities: [[7.3145545e-01 7.0336232e-06 1.6788846e-01 1.0064911e-01]]
Predicted class indices: [0]
Predicted face shape: oval


In [46]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('girlFacevgg13.h5')

# Convert the model to TFLite with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the TFLite model to disk
with open('girlvgg.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite and saved as 'girlvgg3.tflite'")

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp0fbdj9dg\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp0fbdj9dg\assets


Model successfully converted to TFLite and saved as 'girlvgg3.tflite'
